In [16]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp

In [17]:
class OneToOneITU():
    def __init__(self, n, m, parameters , lbs=(0, 0), tol=1e-9):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
        self.tol = tol 
       
        self.A_ij = parameters[0]
        self.B_ij = parameters[1]

    def D_ij(self, U_i, V_j):
        return (U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij)/(1+ self.A_ij)

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, j_idx]

    def get_V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]

In [18]:
def transpose_mkt(self):
    n, m = self.n, self.m
    self.m = n
    self.n = m

    self.A_ij = 1 / self.A_ij.T
    self.B_ij = self.B_ij.T/ self.A_ij

OneToOneITU.transpose_mkt = transpose_mkt

In [19]:
def ITU_parallel_auction(self):

    V_j = np.ones(self.m) * self.lb_V
    mu_ij = np.ones( self.n, dtype= int) * (self.m +1)

    unassigned = np.where(mu_ij == self.m + 1)[0] 
    n_unassigned = len(unassigned)
    iter = 0

    while n_unassigned > 0:# and iter <500000:
        U_ij = self.get_U_ij( V_j, unassigned)
        # print(U_ij)
        perm_unmatched = np.all(U_ij <= self.lb_U, axis = 1)
        
        if np.any(perm_unmatched):
            mu_ij[unassigned] += (self.m - mu_ij[unassigned]) * perm_unmatched
            # print(perm_unmatched)
            # print(mu_ij)
        
        else:
            
            U_ij = np.concatenate((U_ij, self.lb_U * np.ones((n_unassigned,1))), axis = 1)
            U_ij_sorted_id = np.argsort(U_ij, axis = 1)
            j_i = U_ij_sorted_id[:,-1]
            w_i = U_ij[np.arange(n_unassigned),U_ij_sorted_id[:,-2] ]
      
            j_unique = np.unique(j_i)
            #offers = w_i[:,None] * ( j_i[:,None] == j_unique[None,:])  - 1e22 *  ( j_i[:,None] != j_unique[None,:]) 
            #print(np.shape( self.get_V_ij(unassigned[:,None],j_unique[None,:],w_i[:,None])))
            offers = (self.get_V_ij(unassigned[:,None],j_unique[None,:],w_i[:,None]) * ( j_i[:,None] == j_unique[None,:]) 
                       - 1e22 *  ( j_i[:,None] != j_unique[None,:]) )
            i_j_among_unass = np.argmax(offers, axis = 0)
            i_j = unassigned[ i_j_among_unass]
            best_offer_j = np.max(offers, axis = 0)

            # print(i_j)
            #print(offers.round(1))
            

            mu_ij[  np.any( mu_ij[:,None] == j_unique , axis = 1) ] = self.m +1
            mu_ij[i_j] = j_unique
            # print(mu_ij)
            

            #V_j[j_unique] += np.maximum(self.get_V_ij(i_j,j_unique, best_offer_j) - V_j[j_unique], self.tol)
            V_j[j_unique] += np.maximum(best_offer_j - V_j[j_unique], self.tol)
            
        unassigned = np.where(mu_ij == self.m + 1)[0]
        # print(unassigned)
        n_unassigned = len(unassigned)
        iter += 1


    matched_i =  mu_ij < self.m
    U_i = np.ones(self.n) * self.lb_U
    U_i[matched_i] = self.get_U_ij(V_j, matched_i)[np.arange(matched_i.sum()), mu_ij[matched_i]]
    mu_ij = mu_ij[:,None] == np.arange(self.m +1)
    
    

    return mu_ij, U_i, V_j


OneToOneITU.ITU_parallel_auction = ITU_parallel_auction

Generate a a parametrized example

In [42]:
np.random.seed(10)
n_i = 500
m_j = 300

θ_true =  np.array([.5,.5,5,5])
k = len(θ_true)
X_ijk = np.random.normal(1,1, size = [n_i,m_j, k])
 

A = np.exp( X_ijk[:,:,:2] @ θ_true[:2])
B = X_ijk[:,:,2:4] @ θ_true[2:4]

In [43]:
obs_mrk = OneToOneITU( n_i,m_j, (A,B), tol=0.00001)

In [44]:
mu_obs, u_true, v_true= obs_mrk.ITU_parallel_auction()

In [45]:
np.min((u_true[:,None] + obs_mrk.A_ij * v_true[None,:] - obs_mrk.B_ij) / (1+obs_mrk.A_ij))

-2.800990019236473e-15

In [46]:
np.min(obs_mrk.D_ij(u_true, v_true) )

-2.800990019236473e-15

In [47]:
print( np.max(mu_obs[:,-1] * ( u_true - obs_mrk.lb_U)))
print( np.max((mu_obs[:,:-1].sum(0) == 0) * ( v_true - obs_mrk.lb_V)))

0.0
0.0


In [53]:
print(np.all(mu_obs[:,:-1].sum(0) <= 1))
print(np.all(mu_obs[:,:-1].sum(1) <= 1))

True
True


####  Now let us compute the $j$-favourite solution.

In [12]:
obs_mrk.transpose_mkt()
mu_obs_i_fav, v_true_i_fav, u_true_i_fav= obs_mrk.ITU_parallel_auction()
print(np.min(obs_mrk.D_ij(v_true_i_fav, u_true_i_fav) ))
obs_mrk.transpose_mkt()

0.0


In [13]:
# obs_mrk.get_U_ij(u_true_i_fav, np.arange(obs_mrk.n)).round(2)

In [14]:
v_true_i_fav

array([37.904845  , 19.80914613, 45.38516005])

In [15]:
u_true_i_fav

array([  28.21014183,    0.        , 1699.81742942,    0.        ,
        900.51379622])

In [16]:
u_true


array([ 0.00000000e+00,  1.33688387e+01,  0.00000000e+00, -1.77635684e-15,
        0.00000000e+00])

In [17]:
mu_obs_i_fav *1

array([[0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]])

In [18]:
# obs_mrk.get_V_ij(True, True,v_true_i_fav[:,None]).round(0)

In [19]:
# obs_mrk.D_ij(v_true_i_fav, u_true_i_fav) 

In [20]:
mu_obs[:,:-1]*1

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 0],
       [1, 0, 0],
       [0, 1, 0]])

In [21]:
mu_obs_i_fav[:,:-1].T *1

array([[0, 1, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 1]])

In [26]:
obs_mrk.get_U_ij(v_true_i_fav, np.arange(obs_mrk.n))

array([[-284.49051193,  -39.38243347,  -95.11873304],
       [ -92.97558894,  -20.61218004,    4.06047417],
       [-399.5765981 ,  -57.97051162,  -86.38666432],
       [ -93.68838042,  -59.99841921,  -64.31828008],
       [-115.65333297,  -43.34458099, -638.6740028 ]])

In [27]:
np.min(u_true_i_fav - u_true)

-13.368838714775038

# Estimation

In [29]:
def Q(self,θ,mu_obs,X_ijk):
    A_θ = np.exp( X_ijk[:,:,:2] @ θ[:2])
    B_θ = X_ijk[:,:,2:4] @ θ[2:4]

    mkt_θ = OneToOneITU( n_i,m_j, (A_θ,B_θ), tol= .0001)
    mu_θ, u_θ, v_θ = mkt_θ.ITU_parallel_auction()


    Q_θ = ( (mu_obs[:,:-1] * mkt_θ.D_ij(u_θ, v_θ) ).sum() + 
            (mu_obs[:,-1] * u_θ ).sum() + 
            ((mu_obs[:,:-1].sum(0) == 0) * v_θ).sum() +
            np.log( mu_θ.sum()) +
            ((v_true - v_θ)**2).sum())
    


    print(Q_θ)
    print( θ.round(3))
    return Q_θ

OneToOneITU.Q = Q

In [30]:
obs_mrk.Q(np.array([.5,.5,5,5]), mu_obs, X_ijk)

3.912023005428146
[0.5 0.5 5.  5. ]


3.912023005428146

In [39]:
def maximum_score_estimator(self,θ_0,mu_obs,X_ijk ):

    problem = sp.optimize.minimize( lambda θ: self.Q(θ,mu_obs,X_ijk)  ,θ_0)  

    return problem.x


OneToOneITU.maximum_score_estimator = maximum_score_estimator

In [41]:
obs_mrk.maximum_score_estimator( np.array([1,1,4,4]), mu_obs,X_ijk )

1508108.8436401028
[1. 1. 4. 4.]
1508109.9675426218
[1. 1. 4. 4.]
1508108.303221093
[1. 1. 4. 4.]
1508109.0498185747
[1. 1. 4. 4.]
1508108.653116197
[1. 1. 4. 4.]
4130240.476014214
[0.112 1.427 3.837 4.151]
4130240.5486970777
[0.112 1.427 3.837 4.151]
4130240.828511316
[0.112 1.427 3.837 4.151]
4130240.5127621433
[0.112 1.427 3.837 4.151]
4130240.4789089677
[0.112 1.427 3.837 4.151]
787661.2413840991
[0.712 1.138 3.947 4.049]
787659.1763342661
[0.712 1.138 3.947 4.049]
787663.8287009576
[0.712 1.138 3.947 4.049]
787660.9125994297
[0.712 1.138 3.947 4.049]
787661.5707520134
[0.712 1.138 3.947 4.049]
1214591.325618175
[0.914 1.041 3.984 4.015]
1214591.357094702
[0.914 1.041 3.984 4.015]
1214591.512658455
[0.914 1.041 3.984 4.015]
1214591.3517591397
[0.914 1.041 3.984 4.015]
1214591.3124772343
[0.914 1.041 3.984 4.015]
794470.6432685552
[0.889 1.    3.974 4.021]
794470.6878357105
[0.889 1.    3.974 4.021]
794470.7456832285
[0.889 1.    3.974 4.021]
794470.6484489435
[0.889 1.    3.974 4.0

array([0.78114353, 0.91132086, 4.22335615, 3.96998561])